In [2]:
import dask
import h5py

import numpy as np
import dask.array as da

import memory_profiler
%load_ext memory_profiler

In [3]:
Y = da.random.normal(size=(10000, 10000),
                     chunks=(1000, 1000))
Y

dask.array<normal, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

In [4]:
def vectorized_RBF_kernel(X, sigma):
    # % This is equivalent to computing the kernel on every pair of examples
    X2 = np.sum(np.multiply(X, X), 1) # sum colums of the matrix
    K0 = X2 + X2.T - 2 * X * X.T
    K = np.power(np.exp(-1.0 / sigma ** 2), K0)
    return K

def manual_rbf_kernel(X, gamma=0.1):
    X_norm = np.sum(X ** 2, axis = -1)
    K = np.exp(-gamma * (X_norm[:,None] + X_norm[None,:] - 2 * np.dot(X, X.T)))
    return K

In [5]:
def load_features_from_file(path):
    with open(path, 'r') as features_file:
        return [[float(feature) for feature in feature_set.split()] for feature_set in features_file.readlines()]

In [6]:
features_matrix = np.matrix(load_features_from_file(
    '../../features/all_features-forro.txt'
    ))

In [7]:
features_matrix.shape

(27352, 160)

In [9]:
similarity_forro = manual_rbf_kernel(features_matrix, gamma=0.1)

LinAlgError: Last 2 dimensions of the array must be square

In [27]:

def vectorized_RBF_kernel_dask(X, sigma):
    # % This is equivalent to computing the kernel on every pair of examples
    X2 = np.sum(np.multiply(X, X), 1) # sum colums of the matrix
    K0 = X2 + X2.T - 2 * X * X.T
    K = np.power(np.exp(-1.0 / sigma**2), K0)
    return K

In [45]:
similarity_matrix = manual_rbf_kernel(features_matrix.T, gamma=0.1)

LinAlgError: Last 2 dimensions of the array must be square

In [19]:
dask.array.to_hdf5('myfile.hdf5', '/C', x)

In [20]:
f = h5py.File('myfile.hdf5') # HDF5 file
d = f['/C']          # Pointer on on-disk array
d.shape                      # d can be very large


(109269, 109269)

In [21]:
x = da.from_array(d, chunks=(1000, 1000))
x

dask.array<array, shape=(109269, 109269), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>